In [1]:
import os
import csv
import psycopg2
from sqlalchemy import create_engine
from psycopg2 import sql
import pandas as pd
import create_table_leanx as ctl

In [2]:
# ctl.get_raw_query_as_str(table_name='VBAK')

In [3]:
db_host = ''
db_port = ''
db_name = ''
db_user = ''
db_password = ''

# Construct the connection string
connection_string = sql.SQL("dbname={} user={} password={} host={} port={}").format(
    sql.Identifier(db_name),
    sql.Identifier(db_user),
    sql.Identifier(db_password),
    sql.Identifier(db_host),
    sql.Identifier(db_port)
)

In [4]:
connection = psycopg2.connect(f"dbname={db_name} user={db_user} password={db_password} host={db_host} port={db_port}")
cursor = connection.cursor()

cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_name = 'ocpm_om';")

# cursor.execute("SELECT schema_name FROM information_schema.schemata;")
# cursor.execute("""
# SELECT table_name
# FROM information_schema.tables
# WHERE table_type = 'BASE TABLE' AND table_schema = 'celonis_ocpm';
# """)
table_names = cursor.fetchall()
print("Tables in the database:")
for table in table_names:
    print(table[0])

cursor.close()
connection.close()

Tables in the database:


In [5]:
connection = psycopg2.connect(f"dbname={db_name} user={db_user} password={db_password} host={db_host} port={db_port}")
cursor = connection.cursor()
# root_folder = "data/OCPM 011124 1858"
root_folder = "data/OCPM"

def find_csv_files(root_folder):
    csv_files = []
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith(".csv"):
                csv_files.append(os.path.join(root, file))
    return csv_files

def create_or_overwrite_table(csv_file_path, cursor):
    table_name = csv_file_path.split('/')[-1]
    table_name = table_name.split('.')[0]
    print(f"Creating or overwriting table: {table_name} with data from: {csv_file_path}")

    df = pd.read_csv(csv_file_path)
    engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
    df.to_sql(name=table_name, con=engine, schema='ocpm_om', if_exists='replace', index=False)

csv_files = find_csv_files(root_folder)
for csv_file in csv_files:
    create_or_overwrite_table(csv_file, cursor)

cursor.close()
connection.close()

Creating or overwriting table: T006D with data from: data/OCPM/master/T006D.csv
Creating or overwriting table: T134T with data from: data/OCPM/master/T134T.csv
Creating or overwriting table: MARM with data from: data/OCPM/master/MARM.csv
Creating or overwriting table: T001W with data from: data/OCPM/master/T001W.csv
Creating or overwriting table: KNA1 with data from: data/OCPM/master/KNA1.csv
Creating or overwriting table: TVRO with data from: data/OCPM/master/TVRO.csv
Creating or overwriting table: MBEW with data from: data/OCPM/master/MBEW.csv
Creating or overwriting table: USR21 with data from: data/OCPM/master/USR21.csv
Creating or overwriting table: T023T with data from: data/OCPM/master/T023T.csv
Creating or overwriting table: T001 with data from: data/OCPM/master/T001.csv
Creating or overwriting table: ADRP with data from: data/OCPM/master/ADRP.csv
Creating or overwriting table: T001K with data from: data/OCPM/master/T001K.csv
Creating or overwriting table: USR02 with data from: